# arg_scope
使用arg_scope，可以提供一种特殊的命名空间，在这个命名空间中可以为指定的操作设置默认的参数，从而简化代码结构。
以下实例主要实现以下内容：
1. 以普通的方式，实现三个卷积操作；
2. 使用arg_scope等效实现上述例子；
3. 嵌套使用arg_scope
4. 构建VGG16的模型

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [2]:
# 创建输入数据
input = tf.Variable(tf.truncated_normal([1, 320, 320, 3], 
                                        dtype=tf.float32,
                                       stddev=1e-1),
                   name='input')

In [3]:
# 以普通的方式实现卷积操作
normal_net = slim.conv2d(input, 64, [11, 11], 4, 
                         padding='SAME', 
                         weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                         weights_regularizer=slim.l2_regularizer(0.0005),
                         scope='normal_conv1')
normal_net = slim.conv2d(normal_net, 128, [11, 11],
                         padding='VALID',
                         weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                         weights_regularizer=slim.l2_regularizer(0.0005),
                         scope='normal_conv2')
normal_net = slim.conv2d(normal_net, 256, [11, 11],
                         padding='SAME', 
                         weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                         weights_regularizer=slim.l2_regularizer(0.0005),
                         scope='normal_conv3')

In [4]:
# 以arg_scope的方式实现上述卷积操作
with slim.arg_scope([slim.conv2d], padding='SAME', 
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                    weights_regularizer=slim.l2_regularizer(0.0005)):
    arg_net = slim.conv2d(input, 64, [11, 11], scope='arg_conv1')
    arg_net = slim.conv2d(arg_net, 128, [11, 11], padding='VALID', scope='arg_conv2')
    arg_net = slim.conv2d(arg_net, 256, [11, 11], scope='arg_conv3')

In [5]:
# 嵌套使用arg_scope
with slim.arg_scope([slim.conv2d, slim.fully_connected],
                   activation_fn=tf.nn.relu,
                   weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                   weights_regularizer=slim.l2_regularizer(0.0005)):
    with slim.arg_scope([slim.conv2d], stride=1, padding='SAME'):
        net = slim.conv2d(input, 64, [11, 11], padding='VALID', scope='conv1')
        net = slim.conv2d(net, 256, [5, 5], 
                          weights_initializer=tf.truncated_normal_initializer(stddev=0.03),
                         scope='conv2')
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc')

In [6]:
# 构建VGG16的模型
def vgg6(inputs):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                       activation_fn=tf.nn.relu,
                       weigths_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                       weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
        net = slim.max_pool2d(net, [2, 2], scope='pool1')
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
        net = slim.max_pool2d(net, [2, 2], scope='pool2')
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
        net = slim.max_pool2d(net, [2, 2], scope='pool3')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
        net = slim.max_pool2d(net, [2, 2], scope='pool4')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
        net = slim.max_pool2d(net, [2, 2], scope='pool5')
        net = slim.fully_connected(net, 4096, scope='fc6')
        net = slim.dropout(net, 0.5, scope='dropout6')
        net = slim.fully_connected(net, 4096, scope='fc7')
        net = slim.dropout(net, 0.5, scope='dropout7')
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc8')
    return net

In [7]:
# 建立会话
with tf.Session() as sess:
    # 执行变量初始化操作
    sess.run(tf.global_variables_initializer())
    
    # 执行其他操作
    pass